In [8]:
import funcs.data_wrangling as dw
import pandas as pd
import numpy as np
import pyarrow as pa
from dtype_diet import report_on_dataframe, optimize_dtypes
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.express as px
# from matplotlib import rcParams
# from statsmodels.tsa.seasonal import STL
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsforecast import StatsForecast
# from statsforecast.models import MSTL, AutoARIMA

#rcParams['figure.figsize'] = 15, 5

import warnings
warnings.filterwarnings('ignore')

In [5]:
data = dw.ons_data(freq='h', ano_inicio=2000, ano_fim=2023, idreg="S")
df = dw.pipeline(data, update=False)

In [6]:
df.head()

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01 00:00:00,S,SUL,5777.0
2000-01-01 01:00:00,S,SUL,5580.7
2000-01-01 02:00:00,S,SUL,5098.7
2000-01-01 03:00:00,S,SUL,4753.7
2000-01-01 04:00:00,S,SUL,4584.1


In [9]:
proposed_df = report_on_dataframe(df, unit="MB")
new_df = optimize_dtypes(df, proposed_df)
print(f'Original df memory: {df.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 25.94696044921875 MB
Propsed df memory: 3.4857311248779297 MB


In [12]:
df.dtypes

id_reg         object
desc_reg       object
load_mwmed    float64
dtype: object

In [11]:
new_df.dtypes

id_reg        category
desc_reg      category
load_mwmed     float64
dtype: object

In [7]:
my_schema = pa.Schema.from_pandas(df)
my_schema

id_reg: string
desc_reg: string
load_mwmed: double
date: timestamp[ns]
-- schema metadata --
pandas: '{"index_columns": ["date"], "column_indexes": [{"name": null, "f' + 665